In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cystic_Fibrosis/GSE100521'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional and Functional Diversity of Macrophage Repolarization: Implications in Chronic Inflammation and Cystic Fibrosis"
!Series_summary	"Macrophage plasticity allows cells to adopt different phenotypes, a property with potentially important implications in chronic pulmonary disorders such as cystic fibrosis (CF). We examined the transcriptional and functional significance of macrophage repolarization from an “M1” (LPS-stimulated) towards an “M2” phenotype using 5 stimuli. We found that macrophages exhibit highly diverse responses to distinct M2-polarizing stimuli. Specifically, we observed that IL-10 abrogated LPS-tolerance allowing for rapid restoration of LPS responsiveness. In contrast, IL-4 enhanced LPS-tolerance, dampening pro-inflammatory responses after repeat LPS challenge. We found enrichment of phagocytosis-associated pathways in macrophages stimulated with IL-10, leading them to display the greatest efferocytosis ability. Final

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = 1
gender_row = 2

def convert_trait(value):
    if "CF patient" in value:
        return 1
    elif "Non CF subject" in value:
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    gender = value.split(': ')[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE100521', './preprocessed/Cystic_Fibrosis/cohort_info.json', is_gene_available, trait_row is not None) 

selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cystic_Fibrosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Cystic_Fibrosis/trait_data/GSE100521.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM2685867': [0.0, 28.0, 1.0], 'GSM2685868': [0.0, 27.0, 1.0], 'GSM2685869': [0.0, 26.0, 0.0], 'GSM2685870': [0.0, 31.0, 0.0], 'GSM2685871': [0.0, 21.0, 1.0], 'GSM2685872': [0.0, 25.0, 1.0], 'GSM2685873': [0.0, 28.0, 1.0], 'GSM2685874': [0.0, 27.0, 1.0], 'GSM2685875': [0.0, 26.0, 0.0], 'GSM2685876': [0.0, 31.0, 0.0], 'GSM2685877': [0.0, 21.0, 1.0], 'GSM2685878': [0.0, 25.0, 1.0], 'GSM2685879': [0.0, 28.0, 1.0], 'GSM2685880': [0.0, 27.0, 1.0], 'GSM2685881': [0.0, 26.0, 0.0], 'GSM2685882': [0.0, 31.0, 0.0], 'GSM2685883': [0.0, 21.0, 1.0], 'GSM2685884': [0.0, 25.0, 1.0], 'GSM2685885': [0.0, 28.0, 1.0], 'GSM2685886': [0.0, 27.0, 1.0], 'GSM2685887': [0.0, 26.0, 0.0], 'GSM2685888': [0.0, 31.0, 0.0], 'GSM2685889': [0.0, 21.0, 1.0], 'GSM2685890': [0.0, 25.0, 1.0], 'GSM2685891': [0.0, 28.0, 1.0], 'GSM2685892': [0.0, 27.0, 1.0], 'GSM2685893': [0.0, 26.0, 0.0], 'GSM2685894': [0.0, 31.0, 0.0], 'GSM2685895': [0.0, 21.0, 1.0], 'GSM2685896': [0.0, 25.0, 1.0], 'GSM2685897': [0.0, 28.0, 1.0], 'GSM268

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Cystic_Fibrosis/gene_data/GSE100521.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Cystic_Fibrosis')

# 4. Save the cohort information.
save_cohort_info('GSE100521', './preprocessed/Cystic_Fibrosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Cystic_Fibrosis/GSE100521.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Cystic_Fibrosis', the least common label is '0.0' with 36 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Cystic_Fibrosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.75
  50% (Median): 27.0
  75%: 29.0
Min: 21.0
Max: 32.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 24 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

